In [1]:
%pip install -qU langchain-community llama-cpp-python -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 19.6 MB/s eta 0:00:00


In [8]:
!sudo apt-get install git-lfs
!git lfs install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Git LFS initialized.


### https://huggingface.co/docs/huggingface_hub/en/guides/download

In [12]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF", filename="Hermes-2-Pro-Llama-3-8B-Q8_0.gguf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Hermes-2-Pro-Llama-3-8B-Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--NousResearch--Hermes-2-Pro-Llama-3-8B-GGUF/snapshots/fc0d3245797976d0effe82a4f6803a5814e6a4fe/Hermes-2-Pro-Llama-3-8B-Q8_0.gguf'

In [13]:
local_model="/root/.cache/huggingface/hub/models--NousResearch--Hermes-2-Pro-Llama-3-8B-GGUF/snapshots/fc0d3245797976d0effe82a4f6803a5814e6a4fe/Hermes-2-Pro-Llama-3-8B-Q8_0.gguf"

In [14]:
import multiprocessing

from langchain_community.chat_models import ChatLlamaCpp

llm = ChatLlamaCpp(
    temperature=0.5,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=8,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=512,
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.5,
    top_p=0.5,
    verbose=True,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--NousResearch--Hermes-2-Pro-Llama-3-8B-GGUF/snapshots/fc0d3245797976d0effe82a4f6803a5814e6a4fe/Hermes-2-Pro-Llama-3-8B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   

In [15]:
from langchain.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field


class WeatherInput(BaseModel):
    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: str = Field(enum=["celsius", "fahrenheit"])


@tool("get_current_weather", args_schema=WeatherInput)
def get_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    return f"Now the weather in {location} is 22 {unit}"


llm_with_tools = llm.bind_tools(
    tools=[get_weather],
    tool_choice={"type": "function", "function": {"name": "get_current_weather"}},
)

In [16]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in HCMC in celsius",
)

from_string grammar:
char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
location-kv ::= ["] [l] [o] [c] [a] [t] [i] [o] [n] ["] space [:] space string 
space ::= space_7 
string ::= ["] string_8 ["] space 
root ::= [{] space location-kv [,] space unit-kv [}] space 
unit-kv ::= ["] [u] [n] [i] [t] ["] space [:] space unit 
space_7 ::= [ ] | 
string_8 ::= char string_8 | 
unit ::= ["] [c] [e] [l] [s] [i] [u] [s] ["] | ["] [f] [a] [h] [r] [e] [n] [h] [e] [i] [t] ["] 


llama_print_timings:        load time =    7464.95 ms
llama_print_timings:      sample time =     632.82 ms /    29 runs   (   21.82 ms per token,    45.83 tokens per second)
llama_print_timings: prompt eval time =    7464.39 ms /    21 tokens (  355.45 ms per token,     2.81 tokens per second)
llama_print_timings:        eval time =   22640.52 ms /    28 runs   (  808.59 ms per token,     1.24 tokens per second)
llama_print_timings:       total time =   30915.60 ms /  

In [17]:
ai_msg.tool_calls

[{'name': 'get_current_weather',
  'args': {'location': '-10.7646° N, 106.6387 ° E', 'unit': 'celsius'},
  'id': 'call__0_get_current_weather_cmpl-edd2a9b0-964a-4947-9c97-146d830d648d'}]

##Resources:
- https://python.langchain.com/v0.2/docs/integrations/chat/llamacpp/